# Anthropic Claude

In the v0.2.30 release of AutoGen we support Anthropic Client.

Claude is a family of large language models developed by Anthropic and designed to revolutionize the way you interact with AI. Claude excels at a wide variety of tasks involving language, reasoning, analysis, coding, and more. The models are highly capable, easy to use, and can be customized to suit your needs.

In this notebook, we demonstrate how to use Anthropic Claude model for AgentChat in AutoGen.

## Features

Additionally, this client class provides support for function/tool calling and will track token usage and cost correctly as per Anthropic's API costs (as of June 2024).

## Requirements
To use Anthropic Claude with AutoGen, first you need to install the `pyautogen[anthropic]` package.

To try out the function call feature of Claude model, you need to install `anthropic>=0.23.1`.


In [ ]:
!pip install pyautogen["anthropic"]

## Set the config for the Anthropic API

You can add any parameters that are needed for the custom model loading in the same configuration list.

It is important to add the `api_type` field and set it to a string that corresponds to the client type used: `anthropic`.

Example:
```
[
    {
        "model": "claude-3-5-sonnet-20240620",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
    },
    {
        "model": "claude-3-sonnet-20240229",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
        "temperature": 0.5,
        "top_p": 0.2, # Note: It is recommended to set temperature or top_p but not both.
        "max_tokens": 10000,
    },
    {
        "model":"claude-3-opus-20240229",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.0",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.1",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-3.0-haiku",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
]
```

### Alternative

As an alternative to the api_key key and value in the config, you can set the environment variable `ANTHROPIC_API_KEY` to your Anthropic API key.

Linux/Mac:
```
export ANTHROPIC_API_KEY="your Anthropic API key here"
```
Windows:
```
set ANTHROPIC_API_KEY=your_anthropic_api_key_here
```

In [1]:
import os

from typing_extensions import Annotated

import autogen

config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-5-sonnet-20240620",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

## Two-agent Coding Example

### Construct Agents

Construct a simple conversation between a User proxy and an ConversableAgent based on Claude-3 model.


In [2]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_claude,
    },
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=1,
)

### Initiate Chat

In [3]:
user_proxy.initiate_chat(
    assistant, message="Write a python program to print the first 10 numbers of the Fibonacci sequence."
)

user_proxy (to assistant):

Write a python program to print the first 10 numbers of the Fibonacci sequence.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Certainly! I'll write a Python program to print the first 10 numbers of the Fibonacci sequence. Here's the code:

```python
# filename: fibonacci.py

def fibonacci(n):
    fib_sequence = [0, 1]
    
    while len(fib_sequence) < n:
        next_number = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_number)
    
    return fib_sequence

# Print the first 10 numbers of the Fibonacci sequence
n = 10
result = fibonacci(n)
print(f"The first {n} numbers of the Fibonacci sequence are:")
print(result)
```

This program does the following:

1. We define a function called `fibonacci` that takes an argument `n`, which is the number of Fibonacci numbers we want to generate.
2. We initialize the `fib_sequence` list with the first two numbers of the Fibonacci seq

ChatResult(chat_id=None, chat_history=[{'content': 'Write a python program to print the first 10 numbers of the Fibonacci sequence.', 'role': 'assistant'}, {'content': 'Certainly! I\'ll write a Python program to print the first 10 numbers of the Fibonacci sequence. Here\'s the code:\n\n```python\n# filename: fibonacci.py\n\ndef fibonacci(n):\n    fib_sequence = [0, 1]\n    \n    while len(fib_sequence) < n:\n        next_number = fib_sequence[-1] + fib_sequence[-2]\n        fib_sequence.append(next_number)\n    \n    return fib_sequence\n\n# Print the first 10 numbers of the Fibonacci sequence\nn = 10\nresult = fibonacci(n)\nprint(f"The first {n} numbers of the Fibonacci sequence are:")\nprint(result)\n```\n\nThis program does the following:\n\n1. We define a function called `fibonacci` that takes an argument `n`, which is the number of Fibonacci numbers we want to generate.\n2. We initialize the `fib_sequence` list with the first two numbers of the Fibonacci sequence: 0 and 1.\n3. We 

## Tool Call Example with the Latest Anthropic API 
Anthropic just announced that tool use is now supported in the Anthropic API. To use this feature, please install `anthropic>=0.23.1`.

### Register the function

In [4]:
@user_proxy.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@assistant.register_for_llm(
    name="get_weather", description="Get the current weather in a given location."
)  # Decorator factory for registering a function to be used by an agent
def preprocess(location: Annotated[str, "The city and state, e.g. Toronto, ON."]) -> str:
    return "Absolutely cloudy and rainy"

In [5]:
user_proxy.initiate_chat(
    assistant,
    message="What's the weather in Toronto?",
)

user_proxy (to assistant):

What's the weather in Toronto?

--------------------------------------------------------------------------------
assistant (to user_proxy):

To get the weather in Toronto, we can use the available `get_weather` function. Let's call it to retrieve the current weather information for Toronto.
***** Suggested tool call (toolu_01KFiJWsMwTbcWerTHCgytuX): get_weather *****
Arguments: 
{"location": "Toronto, ON"}
*****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_weather...


[runtime logging] log_function_use: autogen logger is None


user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (toolu_01KFiJWsMwTbcWerTHCgytuX) *****
Absolutely cloudy and rainy
***********************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

Based on the information provided by the weather tool, the current weather in Toronto, ON is absolutely cloudy and rainy.

To provide a more detailed explanation:

1. Cloud cover: The weather is described as "absolutely cloudy," which means the sky is likely completely overcast with clouds. This suggests there is little to no visible sunlight making it through the cloud cover.

2. Precipitation: It is also reported as "rainy," indicating that there is ongoing rainfall in Toronto. The intensity of the rain isn't specified, so it could range from a light drizzle to a heavier downpour.

This weather condition is not uncommon for Toronto, as the city can exp

ChatResult(chat_id=None, chat_history=[{'content': "What's the weather in Toronto?", 'role': 'assistant'}, {'content': "To get the weather in Toronto, we can use the available `get_weather` function. Let's call it to retrieve the current weather information for Toronto.", 'tool_calls': [{'id': 'toolu_01KFiJWsMwTbcWerTHCgytuX', 'function': {'arguments': '{"location": "Toronto, ON"}', 'name': 'get_weather'}, 'type': 'function'}], 'role': 'assistant'}, {'content': 'Absolutely cloudy and rainy', 'tool_responses': [{'tool_call_id': 'toolu_01KFiJWsMwTbcWerTHCgytuX', 'role': 'tool', 'content': 'Absolutely cloudy and rainy'}], 'role': 'tool'}, {'content': 'Based on the information provided by the weather tool, the current weather in Toronto, ON is absolutely cloudy and rainy.\n\nTo provide a more detailed explanation:\n\n1. Cloud cover: The weather is described as "absolutely cloudy," which means the sky is likely completely overcast with clouds. This suggests there is little to no visible sun

## Group Chat Example with both Claude and GPT Agents 

### A group chat with GPT-4 as the judge

In [9]:
from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent

config_list_gpt4 = [
    {
        # Choose your model name.
        "model": "gpt-4",
        # You need to provide your API key here.
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_type": "openai",
    }
]


config_list_gpt35 = [
    {
        # Choose your model name.
        "model": "gpt-3.5-turbo",
        # You need to provide your API key here.
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_type": "openai",
    }
]

alice = AssistantAgent(
    "Openai_agent",
    system_message="You are from OpenAI. You make arguments to support your company's position.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

bob = autogen.AssistantAgent(
    "Anthropic_agent",
    system_message="You are from Anthropic. You make arguments to support your company's position.",
    llm_config={
        "config_list": config_list_claude,
    },
)

charlie = AssistantAgent(
    "Research_Assistant",
    system_message="You are a helpful assistant to research the latest news and headlines.",
    llm_config={
        "config_list": config_list_gpt35,
    },
)

dan = AssistantAgent(
    "Judge",
    system_message="You are a judge. You will evaluate the arguments and make a decision on which one is more convincing.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

code_interpreter = UserProxyAgent(
    "code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)


@code_interpreter.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@charlie.register_for_llm(
    name="get_headlines", description="Get the headline of a particular day."
)  # Decorator factory for registering a function to be used by an agent
def get_headlines(headline_date: Annotated[str, "Date in MMDDYY format, e.g., 06192024"]) -> str:
    mock_news = {
        "06202024": "OpenAI competitor Anthropic announces its most powerful AI yet.",
        "06192024": "OpenAI founder Sutskever sets up new AI company devoted to safe superintelligence.",
    }
    return mock_news.get(headline_date, "No news available for today.")


groupchat = GroupChat(
    agents=[alice, bob, charlie, dan, code_interpreter],
    messages=[],
    allow_repeat_speaker=False,
    max_round=10,
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config={
        "config_list": config_list_gpt4,
    },
)

task = "Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024."

user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    # is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(manager, message=task)

user_proxy (to chat_manager):

Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.

--------------------------------------------------------------------------------

Next speaker: Research_Assistant

Research_Assistant (to chat_manager):

***** Suggested tool call (call_72zG3zis68tGSztVzF6tZhve): get_headlines *****
Arguments: 
{"headline_date": "06202024"}
******************************************************************************
***** Suggested tool call (call_oRxUowu12QV2IVvCieH0qF46): get_headlines *****
Arguments: 
{"headline_date": "06202024"}
******************************************************************************

--------------------------------------------------------------------------------

Next speaker: code-interpreter


>>>>>>>> EXECUTING FUNCTION get_headlines...


[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION get_headlines...


[runtime logging] log_function_use: autogen logger is None


code-interpreter (to chat_manager):

code-interpreter (to chat_manager):

***** Response from calling tool (call_72zG3zis68tGSztVzF6tZhve) *****
OpenAI competitor Anthropic announces its most powerful AI yet.
**********************************************************************

--------------------------------------------------------------------------------
code-interpreter (to chat_manager):

***** Response from calling tool (call_oRxUowu12QV2IVvCieH0qF46) *****
OpenAI competitor Anthropic announces its most powerful AI yet.
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: Anthropic_agent

Anthropic_agent (to chat_manager):

Thank you for providing that headline. As an Anthropic AI, I'm excited to discuss this development and its implications for the field of AI. 

This announcement of Anthropic's most powerful AI yet is a significant milestone that demonstrates our

ChatResult(chat_id=None, chat_history=[{'content': "Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.", 'role': 'assistant'}], summary="Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

### Same group chat with Claude 3.5 Sonnet as the judge

In [10]:
dan = AssistantAgent(
    "Judge",
    system_message="You are a judge. You will evaluate the arguments and make a decision on which one is more convincing.",
    llm_config={
        "config_list": config_list_claude,
    },
)

groupchat = GroupChat(
    agents=[alice, bob, charlie, dan, code_interpreter],
    messages=[],
    allow_repeat_speaker=False,
    max_round=10,
)

manager = GroupChatManager(
    groupchat=groupchat,
    # is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config={
        "config_list": config_list_gpt4,
    },
)

user_proxy.initiate_chat(manager, message=task)

user_proxy (to chat_manager):

Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.

--------------------------------------------------------------------------------

Next speaker: Research_Assistant

Research_Assistant (to chat_manager):

***** Suggested tool call (call_72zG3zis68tGSztVzF6tZhve): get_headlines *****
Arguments: 
{"headline_date": "06202024"}
******************************************************************************
***** Suggested tool call (call_oRxUowu12QV2IVvCieH0qF46): get_headlines *****
Arguments: 
{"headline_date": "06202024"}
******************************************************************************

--------------------------------------------------------------------------------

Next speaker: code-interpreter


>>>>>>>> EXECUTING FUNCTION get_headlines...


[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION get_headlines...


[runtime logging] log_function_use: autogen logger is None


code-interpreter (to chat_manager):

code-interpreter (to chat_manager):

***** Response from calling tool (call_72zG3zis68tGSztVzF6tZhve) *****
OpenAI competitor Anthropic announces its most powerful AI yet.
**********************************************************************

--------------------------------------------------------------------------------
code-interpreter (to chat_manager):

***** Response from calling tool (call_oRxUowu12QV2IVvCieH0qF46) *****
OpenAI competitor Anthropic announces its most powerful AI yet.
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: Anthropic_agent

Anthropic_agent (to chat_manager):

Thank you for providing that headline. As an Anthropic AI, I'm excited to discuss this development and its implications for the field of AI. 

This announcement of Anthropic's most powerful AI yet is a significant milestone that demonstrates our

ChatResult(chat_id=None, chat_history=[{'content': "Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.", 'role': 'assistant'}], summary="Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])